<a href="https://colab.research.google.com/github/dgg32/spark-udemy/blob/main/mapreduce_with_mrjob.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mrjob

     |████████████████████████████████| 440kB 2.5MB/s 


In [ ]:
%%file friendcount.py
from mrjob.job import MRJob


class FriendCount(MRJob):
    def mapper(self, _, line):
      (ID, name, age, numFriends) = line.split(",")
      yield age, float(numFriends)
      
    def reducer(self, age, numFriends):
        total_num = 0
        total = 0
        for i in numFriends:
            total += i
            total_num += 1
        ### cannot use len, sum, because numFriends is generator and len, sum will iterate it twice
        ### TypeError: object of type 'generator' has no len()

        #yield age, sum(numFriends)/total_num  not work
        #yield age, total/total_num
        yield age, total/total_num

if __name__ == '__main__':
     FriendCount.run()

Writing friendcount.py


In [ ]:
!python friendcount.py fakefriends.csv > output.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Traceback (most recent call last):
  File "friendcount.py", line 23, in <module>
    FriendCount.run()
  File "/usr/local/lib/python3.6/dist-packages/mrjob/job.py", line 616, in run
    cls().execute()
  File "/usr/local/lib/python3.6/dist-packages/mrjob/job.py", line 687, in execute
    self.run_job()
  File "/usr/local/lib/python3.6/dist-packages/mrjob/job.py", line 636, in run_job
    runner.run()
  File "/usr/local/lib/python3.6/dist-packages/mrjob/runner.py", line 500, in run
    self._check_input_paths()
  File "/usr/local/lib/python3.6/dist-packages/mrjob/runner.py", line 1133, in _check_input_paths
    self._check_input_path(path)
  File "/usr/local/lib/python3.6/dist-packages/mrjob/runner.py", line 1147, in _check_input_path
    'Input path %s does not exist!' % (path,))
OSError: Input path fakefriends.csv does not exist!


In [ ]:
%%file extremet.py
from mrjob.job import MRJob


class ExtremeTemperature(MRJob):
    def MakeFahrenheit(self, tenthsOfCelsius):
        celsius = float(tenthsOfCelsius) / 10.0
        fahrenheit = celsius * 1.8 + 32
        return fahrenheit

    def mapper(self, _, line):
        (location, date, type, data, x, y, z, w) = line.split(",")
        if type == "TMIN":
            temperature = self.MakeFahrenheit(data)

            yield location, temperature
        

    def reducer(self, location, temps):
        
        yield location, min(temps)  
        #yield age, sum(numFriends)   not work

if __name__ == '__main__':
     ExtremeTemperature.run()

Overwriting extremet.py


In [ ]:
!python extremet.py 1800.csv

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/extremet.root.20201029.123719.006536
Running step 1 of 1...
job output is in /tmp/extremet.root.20201029.123719.006536/output
Streaming final output from /tmp/extremet.root.20201029.123719.006536/output...
"EZE00100082"	7.699999999999999
"ITE00100554"	5.359999999999999
Removing temp directory /tmp/extremet.root.20201029.123719.006536...


In [ ]:
%%file book.py
from mrjob.job import MRJob
import re

class Book(MRJob):


    def mapper(self, _, line):
        words = re.split(r'\W+', line)

        for word in words:
            yield word.lower(), 1
        

    def reducer(self, word, values):
        
        yield word, sum(values)  
        #yield age, sum(numFriends)   not work

if __name__ == '__main__':
     Book.run()

Writing book.py


In [ ]:
!python book.py Book > output.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Traceback (most recent call last):
  File "book.py", line 20, in <module>
    Book.run()
  File "/usr/local/lib/python3.6/dist-packages/mrjob/job.py", line 616, in run
    cls().execute()
  File "/usr/local/lib/python3.6/dist-packages/mrjob/job.py", line 687, in execute
    self.run_job()
  File "/usr/local/lib/python3.6/dist-packages/mrjob/job.py", line 636, in run_job
    runner.run()
  File "/usr/local/lib/python3.6/dist-packages/mrjob/runner.py", line 500, in run
    self._check_input_paths()
  File "/usr/local/lib/python3.6/dist-packages/mrjob/runner.py", line 1133, in _check_input_paths
    self._check_input_path(path)
  File "/usr/local/lib/python3.6/dist-packages/mrjob/runner.py", line 1147, in _check_input_path
    'Input path %s does not exist!' % (path,))
OSError: Input path Book does not exist!


In [ ]:
### sort the count

%%file sort_count_book.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import re

class Book(MRJob):

    def steps(self):
        return [
                MRStep(mapper=self.mapper_get_words, reducer=self.reducer_sum_word_counts),
                MRStep(reducer=self.reduce_sort_counts)

        ]
    def mapper_get_words(self, _, line):
        words = re.split(r'\W+', line)

        for word in words:
            if word != "":
                yield word.lower(), 1
        

    def reducer_sum_word_counts(self, key, values):
        yield None, (sum(values), key)
        #yield age, sum(numFriends)   not work
    
    def reduce_sort_counts(self, _, word_counts):
        ### 0001: [I, you, he] ... that is why for loop
        for count, key in sorted(word_counts, reverse=True):
            yield ('%05d' % int(count), key)


if __name__ == '__main__':
     Book.run()

Overwriting sort_count_book.py


In [ ]:
!python sort_count_book.py Book

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/sort_count_book.root.20201029.143636.227548
Running step 1 of 2...
Running step 2 of 2...
job output is in /tmp/sort_count_book.root.20201029.143636.227548/output
Streaming final output from /tmp/sort_count_book.root.20201029.143636.227548/output...
"01878"	"you"
"01828"	"to"
"01420"	"your"
"01292"	"the"
"01191"	"a"
"00970"	"of"
"00934"	"and"
"00747"	"that"
"00649"	"it"
"00616"	"in"
"00560"	"is"
"00537"	"for"
"00428"	"on"
"00424"	"are"
"00411"	"if"
"00391"	"s"
"00387"	"i"
"00383"	"business"
"00376"	"can"
"00369"	"be"
"00343"	"as"
"00321"	"have"
"00315"	"with"
"00301"	"t"
"00280"	"this"
"00278"	"or"
"00255"	"time"
"00242"	"but"
"00234"	"they"
"00231"	"will"
"00229"	"what"
"00220"	"at"
"00215"	"my"
"00214"	"re"
"00207"	"do"
"00203"	"not"
"00202"	"about"
"00200"	"more"
"00182"	"product"
"00178"	"an"
"00177"	"up"
"00174"	"need"
"00166"	"them"
"00166"	"from"
"00163"	"how"

In [ ]:
### sort the count

%%file spending_sort.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import re

class spending_sort(MRJob):

    def steps(self):
        return [
                MRStep(mapper=self.mapper_get_spend, reducer=self.reducer_sum_spend),
                MRStep(reducer=self.reducer_sort_spend)

        ]
    def mapper_get_spend(self, _, line):
        (customer, item, order_amount) = line.split(",")
        yield customer, float(order_amount)
        

    def reducer_sum_spend(self, key, values):
        yield None, (sum(values), key)
        #yield age, sum(numFriends)   not work
    
    def reducer_sort_spend(self, _, spend_customer):
        ### 0001: [I, you, he] ... that is why for loop
        for spend, customer in sorted(spend_customer, reverse=True):
            yield ('%05d' % int(spend), customer)


if __name__ == '__main__':
     spending_sort.run()

Writing spending_sort.py


In [ ]:
!python spending_sort.py customer-orders.csv > output.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Traceback (most recent call last):
  File "spending_sort.py", line 29, in <module>
    spending_sort.run()
  File "/usr/local/lib/python3.6/dist-packages/mrjob/job.py", line 616, in run
    cls().execute()
  File "/usr/local/lib/python3.6/dist-packages/mrjob/job.py", line 687, in execute
    self.run_job()
  File "/usr/local/lib/python3.6/dist-packages/mrjob/job.py", line 636, in run_job
    runner.run()
  File "/usr/local/lib/python3.6/dist-packages/mrjob/runner.py", line 500, in run
    self._check_input_paths()
  File "/usr/local/lib/python3.6/dist-packages/mrjob/runner.py", line 1133, in _check_input_paths
    self._check_input_path(path)
  File "/usr/local/lib/python3.6/dist-packages/mrjob/runner.py", line 1147, in _check_input_path
    'Input path %s does not exist!' % (path,))
OSError: Input path customer-orders.csv does not exist!


# combiner

In [ ]:
%%file combiner_book.py
from mrjob.job import MRJob
import re

class Combiner_Book(MRJob):


    def mapper(self, _, line):
        words = re.split(r'\W+', line)

        for word in words:
            yield word.lower(), 1
    
    def combiner(self, word, values):
        yield word, sum(values)

    def reducer(self, word, values):
        
        yield word, sum(values)  
        #yield age, sum(numFriends)   not work

if __name__ == '__main__':
     Combiner_Book.run()

Writing combiner_book.py


In [ ]:
!python combiner_book.py Book

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/combiner_book.root.20201030.100345.998052
Running step 1 of 1...
job output is in /tmp/combiner_book.root.20201030.100345.998052/output
Streaming final output from /tmp/combiner_book.root.20201030.100345.998052/output...
"didn"	15
"die"	2
"dies"	1
"difference"	4
"different"	21
"differentiate"	1
"differently"	1
"difficult"	7
"difficulty"	1
"dig"	3
"digest"	1
"digging"	2
"digital"	11
"dilemma"	1
"diligence"	1
"diligent"	1
"dimensions"	1
"diminish"	1
"dire"	1
"direct"	13
"directing"	1
"direction"	2
"directly"	11
"director"	2
"directories"	1
"directors"	1
"directx"	1
"disability"	2
"disable"	1
"disappearance"	1
"disappears"	2
"discarding"	1
"discipline"	4
"disciplines"	2
"disclaimer"	2
"discontinue"	1
"discount"	2
"discover"	5
"discovered"	2
"discovers"	1
"discretionary"	1
"discuss"	1
"discussed"	5
"discussion"	1
"discussions"	4
"disease"	1
"display"	18
"disposable"	1
"d

In [ ]:
### sort the count

%%file most_watch.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import re

class MostWatchMovie(MRJob):

    def steps(self):
        return [
                MRStep(mapper=self.mapper_get_movie, reducer=self.reducer_sum_movie),
                MRStep(reducer=self.reducer_most_watch_movie)

        ]
    def mapper_get_movie(self, _, line):
        (userId, movieId, rating, timestamp) = line.split(",")
        if userId != "userId":
            yield movieId, 1
        

    def reducer_sum_movie(self, key, values):
        yield None, (sum(values), key)
        #yield age, sum(numFriends)   not work
    
    def reducer_most_watch_movie(self, _, watch_movie):
        
        (watch, movie) = max(watch_movie)
        yield movie, watch


if __name__ == '__main__':
     MostWatchMovie.run()

Writing most_watch.py


In [ ]:
!python most_watch.py rating.csv

No configs found; falling back on auto-configuration
No configs specified for inline runner
Traceback (most recent call last):
  File "most_watch.py", line 30, in <module>
    MostWatchMovie.run()
  File "/usr/local/lib/python3.6/dist-packages/mrjob/job.py", line 616, in run
    cls().execute()
  File "/usr/local/lib/python3.6/dist-packages/mrjob/job.py", line 687, in execute
    self.run_job()
  File "/usr/local/lib/python3.6/dist-packages/mrjob/job.py", line 636, in run_job
    runner.run()
  File "/usr/local/lib/python3.6/dist-packages/mrjob/runner.py", line 500, in run
    self._check_input_paths()
  File "/usr/local/lib/python3.6/dist-packages/mrjob/runner.py", line 1133, in _check_input_paths
    self._check_input_path(path)
  File "/usr/local/lib/python3.6/dist-packages/mrjob/runner.py", line 1147, in _check_input_path
    'Input path %s does not exist!' % (path,))
OSError: Input path rating.csv does not exist!


In [ ]:
### sort the count

%%file MostPopularSuperHero.py
from mrjob.job import MRJob
from mrjob.step import MRStep


class MostPopularSuperHero(MRJob):

    def configure_args(self):
        super(MostPopularSuperHero, self).configure_args()
        self.add_file_arg('--names', help='Path to Marvel-names.txt')

    def steps(self):
        return [
                MRStep(mapper = self.mapper_count_friends_per_line, reducer = self.reducer_hero_numFriend),
                MRStep(
                       mapper = self.prep_for_sort,
                       mapper_init = self.load_name_dictionary,
                       reducer = self.reducer_most_popular_hero
                      )

        ]
    def mapper_count_friends_per_line(self, _, line):
        fields = line.split()
        heroID = fields[0]
        numFriends = len(fields) - 1
        yield int(heroID), int(numFriends)

    def reducer_hero_numFriend(self, heroID, numFriends):
        ##### there may be replicate in the file
        ##### numFriends is in the form of [3] or [3, 5]
        yield heroID, sum(numFriends)
    
    def prep_for_sort(self, heroID, numFriends):
        yield None, (numFriends, self.heroNames[heroID])

    def reducer_most_popular_hero(self, _, num_hero):
        
        #(friendCounts, hero) = max(num_hero)
        yield max(num_hero)

    def load_name_dictionary(self):
        self.heroNames = {}

        with open(self.options.names, encoding='ascii', errors='ignore') as f:
            for line in f:
                fields = line.split('"')
                heroID = int(fields[0])
                self.heroNames[heroID] = fields[1]

if __name__ == '__main__':
     MostPopularSuperHero.run()

Writing MostPopularSuperHero.py


In [ ]:
!python MostPopularSuperHero.py --names Marvel_Names Marvel_Graph

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/MostPopularSuperHero.root.20201102.110337.961185
Running step 1 of 2...
Running step 2 of 2...
job output is in /tmp/MostPopularSuperHero.root.20201102.110337.961185/output
Streaming final output from /tmp/MostPopularSuperHero.root.20201102.110337.961185/output...
1933	"CAPTAIN AMERICA"
Removing temp directory /tmp/MostPopularSuperHero.root.20201102.110337.961185...


Breadth first search with mapreduce

degree of separation

input format:

id|friends|distance|color

example:

2764|,,,,,,,,4726,2960,3023,3495,867,4024,5072,3329,4473,4704,6000,5925,3250,3720,2664,4441,4981|3|BLACK
2765|,1165,4898,154,2297,3757,6148,491,875,859,1988,6066,5868,3188,5388,2931,5555,4441,5736,5776,6313,5978,1000,,,,,,,,,,,,,,,|3|BLACK

In [ ]:
%%file bfs.py

from mrjob.job import MRJob
from mrjob.protocol import RawValueProtocol

class Node:
    def __init__(self):
        self.characterID = ''
        self.connections = []
        self.distance = 9999
        self.color = 'WHITE'

    #Format is ID|EDGES|DISTANCE|COLOR
    def fromLine(self, line):
        fields = line.split('|')
        if (len(fields) == 4):
            self.characterID = fields[0]
            self.connections = fields[1].split(',')
            self.distance = int(fields[2])
            self.color = fields[3]

    def getLine(self):
        connections = ','.join(self.connections)
        return '|'.join( (self.characterID, connections, str(self.distance), self.color) )

class MRBFSIteration(MRJob):

    INPUT_PROTOCOL = RawValueProtocol
    OUTPUT_PROTOCOL = RawValueProtocol

    def configure_args(self):
        super(MRBFSIteration, self).configure_args()
        self.add_passthru_arg(
            '--target', help="ID of character we are searching for")

    def mapper(self, _, line):
        node = Node()
        node.fromLine(line)
        #If this node needs to be expanded...
        if (node.color == 'GRAY'):
            for connection in node.connections:
                vnode = Node()
                vnode.characterID = connection
                vnode.distance = int(node.distance) + 1
                vnode.color = 'GRAY'
                if (self.options.target == connection):

                    #### Counters here is used to print results.
                    #### mrjob cannot print ()
                    #### the counter value has nothing to do with the distance
                    ####counter consists of group, name and enum value
                    counterName = ("Target ID " + connection +
                        " was hit with distance " + str(vnode.distance))
                    self.increment_counter('Degrees of Separation',
                        counterName, 1)
                yield connection, vnode.getLine()

            #We've processed this node, so color it black
            node.color = 'BLACK'

        #Emit the input node so we don't lose it.
        yield node.characterID, node.getLine()

    def reducer(self, key, values):
        edges = []
        distance = 9999
        color = 'WHITE'

        for value in values:
            node = Node()
            node.fromLine(value)

            if (len(node.connections) > 0):
                #edges = node.connections
                edges.extend(node.connections)

            if (node.distance < distance):
                distance = node.distance

            if ( node.color == 'BLACK' ):
                color = 'BLACK'

            if ( node.color == 'GRAY' and color == 'WHITE' ):
                color = 'GRAY'

        node = Node()
        node.characterID = key
        node.distance = distance
        node.color = color
        #There's a bug in mrjob for Windows where sorting fails
        #with too much data. As a workaround, we're limiting the
        #number of edges to 500 here. You'd remove the [:500] if you
        #were running this for real on a Linux cluster.
        node.connections = edges[:500]

        yield key, node.getLine()

if __name__ == '__main__':
    MRBFSIteration.run()


Writing bfs.py


In [ ]:
!python bfs.py --target 2 test.txt 

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/bfs.root.20201102.231951.362687
Running step 1 of 1...

Counters: 1
	Degrees of Separation
		Target ID 2 was hit with distance 2=1

job output is in /tmp/bfs.root.20201102.231951.362687/output
Streaming final output from /tmp/bfs.root.20201102.231951.362687/output...
1|2|1|BLACK
2|,1,3|2|GRAY
3|2|9999|WHITE
Removing temp directory /tmp/bfs.root.20201102.231951.362687...


In [ ]:
!python bfs.py --target 3 test_2.txt 

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/bfs.root.20201102.231848.614925
Running step 1 of 1...

Counters: 1
	Degrees of Separation
		Target ID 3 was hit with distance 3=1

job output is in /tmp/bfs.root.20201102.231848.614925/output
Streaming final output from /tmp/bfs.root.20201102.231848.614925/output...
||3|GRAY
1|2,|1|BLACK
2|,1,3|2|BLACK
3|,2|3|GRAY
Removing temp directory /tmp/bfs.root.20201102.231848.614925...


In [ ]:
%%file movie_similarities.py


# To run locally:
# !python MovieSimilarities.py --items=ml-100k/u.item ml-100k/u.data > sims.txt

# To run on a single EMR node:
# !python MovieSimilarities.py -r emr --items=ml-100k/u.item ml-100k/u.data

# To run on 4 EMR nodes:
#!python MovieSimilarities.py -r emr --num-ec2-instances=4 --items=ml-100k/u.item ml-100k/u.data

# Troubleshooting EMR jobs (subsitute your job ID):
# !python -m mrjob.tools.emr.fetch_logs --find-failure j-1NXMMBNEQHAFT

from mrjob.job import MRJob
from mrjob.step import MRStep
from math import sqrt

from itertools import combinations

class MovieSimilarities(MRJob):

    def configure_args(self):
        super(MovieSimilarities, self).configure_args()
        self.add_file_arg('--items', help='Path to u.item')
    
    def load_movie_names(self):
        # load database of movie names

        self.movieNames = {}

        header = False
        with open(self.options.items, encoding="ascii", errors="ignore") as f:
            for line in f:
                if header == False:
                    header = True
                else:
                    fields = line.split(",")
                    self.movieNames[int(fields[0])] = fields[1]
    
    def steps(self):

        return [
                MRStep (mapper=self.mapper_parse_input, reducer = self.reducer_ratings_by_user),
                MRStep (mapper=self.mapper_create_item_pairs, reducer=self.reducer_compute_similarity),
                MRStep (mapper=self.mapper_sort_similarities, mapper_init=self.load_movie_names, reducer=self.reducer_output_similarities)
        ]
    
    def mapper_parse_input (self, key, line):
        # Outputs userID => (movieID, rating)
        (userID, movieID, rating, timestamp) = line.split(",")
        if len(line.strip()) > 0 and userID != "userId":
            yield userID, (movieID, float(rating))

    def reducer_ratings_by_user (self, userID, movieRatings):
        # group (movie, rating) pairs by userID

        ratings = []
        for movieID, rating in movieRatings:
            ratings.append((movieID, rating))
        
        yield userID, ratings
    
    def mapper_create_item_pairs(self, userID, movieRatings):
        # Find every pair of movies each user has seen, and emit
        # each pair with its associated ratings

        # "combinations" finds every possible pair from the list of movies
        # this user viewed.
        for itemRating1, itemRating2 in combinations(movieRatings, 2):
            movieID1 = itemRating1[0]
            rating1 = itemRating1[1]
            movieID2 = itemRating2[0]
            rating2 = itemRating2[1]

            yield (movieID1, movieID2), (rating1, rating2)
            yield (movieID2, movieID1), (rating2, rating1)
    
    def cosine_similarity(self, ratingPairs):
        # Computes the cosine similarity metric between two
        # rating vectors.
        numPairs = 0
        sum_xx = sum_yy = sum_xy = 0

        for ratingX, ratingY in ratingPairs:
            sum_xx += ratingX * ratingX
            sum_yy += ratingY * ratingY
            sum_xy += ratingX * ratingY

            numPairs += 1

        numerator = sum_xy
        denominator = sqrt(sum_xx) * sqrt(sum_yy)

        score = 0
        if denominator:
            score = (numerator / float(denominator))
        
        return (score, numPairs)

    def reducer_compute_similarity(self, moviePairs, ratingPairs):
        # Compute the similarity score between the ratings vectors
        # for each movie pair viewed by multiple people

        # Output movie pair => score, number of co-ratings

        score, numPairs = self.cosine_similarity(ratingPairs)

        if numPairs > 10 and score > 0.95:
            yield moviePairs, (score, numPairs)

    def mapper_sort_similarities(self, moviePair, scores):
        # Shuffle things around so the key is (movie1, score)
        # so we have meaningfully sorted results.

        score, n = scores
        movie1, movie2 = moviePair

        yield (self.movieNames[int(movie1)], score), (self.movieNames[int(movie2)], n)
    
    def reducer_output_similarities(self, movieScore, similarN):
        # Output the results.
        # Movie => Similar Movie, score, number of co-ratings
        movie1, score = movieScore
        for movie2, n in similarN:
            yield movie1, (movie2, score, n)

if __name__ == "__main__":
    MovieSimilarities.run()

Overwriting movie_similarities.py


In [ ]:
!python movie_similarities.py --items=movies.csv ratings.csv

Streaming output truncated to the last 5000 lines.
"\"A-Team"	["Slumdog Millionaire (2008)", 0.9779685257819198, 11]
"\"A-Team"	["Harry Potter and the Chamber of Secrets (2002)", 0.9781088796973073, 11]
"\"A-Team"	["Ice Age (2002)", 0.9783649020482809, 11]
"\"A-Team"	["\"Last Samurai", 0.9784528184941126, 11]
"\"A-Team"	["\"Dark Knight Rises", 0.9786105067596504, 11]
"\"A-Team"	["Ratatouille (2007)", 0.9786647901074714, 13]
"\"A-Team"	["I Am Legend (2007)", 0.9787158256403785, 11]
"\"A-Team"	["\"Mask", 0.9789084006475309, 12]
"\"A-Team"	["Ocean's Eleven (2001)", 0.9789915276231436, 14]
"\"A-Team"	["\"Bug's Life", 0.9790800702665309, 11]
"\"A-Team"	["Indiana Jones and the Last Crusade (1989)", 0.9799852358255727, 12]
"\"A-Team"	["\"I", 0.9803504363989919, 13]
"\"A-Team"	["Die Hard (1988)", 0.9808140923860226, 13]
"\"A-Team"	["Seven (a.k.a. Se7en) (1995)", 0.9808535344808531, 11]
"\"A-Team"	["Iron Man (2008)", 0.9808744707718187, 13]
"\"A-Team"	["Star Wars: Episode V - The Empire Strikes

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 0.989569485067437, 17]
"Awakenings (1990)"	["When Harry Met Sally... (1989)", 0.9897078536177618, 13]
"Awakenings (1990)"	["Dead Poets Society (1989)", 0.9897954720365244, 14]
"Awakenings (1990)"	["Kill Bill: Vol. 2 (2004)", 0.9905760964464103, 12]
"Awakenings (1990)"	["\"Silence of the Lambs", 0.9909049210382014, 21]
"Awakenings (1990)"	["\"Sting", 0.9919838847029161, 11]
"Awakenings (1990)"	["What About Bob? (1991)", 0.9921585159173874, 11]
"Awakenings (1990)"	["\"Few Good Men", 0.9928946214458074, 14]
"Awakenings (1990)"	["\"Untouchables", 0.9945403206124314, 15]
"Awakenings (1990)"	["Jackie Brown (1997)", 0.994972045036519, 11]
"Awakenings (1990)"	["\"Shining", 0.9951747050335896, 15]
"Awakenings (1990)"	["\"Others", 0.9962224278706331, 11]
"BASEketball (1998)"	["Jurassic Park (1993)", 0.9508894657309656, 11]
"BASEketball (1998)"	["Star Wars: Episode IV - A New Hope (1977)", 0.9512726488840123, 13]
"BASEketball (1998)"	["Jumanji (1995)", 0.9528518708253985, 12]
"BASEketball (1998)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Streaming output truncated to the last 5000 lines.
"Clerks (1994)"	["Toy Story (1995)", 0.9556873716498808, 59]
"Clerks (1994)"	["\"Gods Must Be Crazy", 0.9558323488653002, 11]
"Clerks (1994)"	["Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)", 0.9558677043552135, 57]
"Clerks (1994)"	["Gattaca (1997)", 0.9559120743334031, 36]
"Clerks (1994)"	["Final Fantasy: The Spirits Within (2001)", 0.955925422927408, 15]
"Clerks (1994)"	["\"Game", 0.95593396316298, 30]
"Clerks (1994)"	["\"History of Violence", 0.9559404987980745, 15]
"Clerks (1994)"	["Who Framed Roger Rabbit? (1988)", 0.9559406562790718, 45]
"Clerks (1994)"	["Leaving Las Vegas (1995)", 0.9560518070420845, 30]
"Clerks (1994)"	["House of Flying Daggers (Shi mian mai fu) (2004)", 0.9561224481132399, 20]
"Clerks (1994)"	["Babe (1995)", 0.956160389234257, 43]
"Clerks (1994)"	["Star Wars: Episode IV - A New Hope (1977)", 0.9561653796590985, 66]
"Clerks (1994)"	["Children of Men (2006)", 0.9562188090193842, 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Streaming output truncated to the last 5000 lines.
"Outbreak (1995)"	["Star Trek: Generations (1994)", 0.9579365405229132, 55]
"Outbreak (1995)"	["Austin Powers: The Spy Who Shagged Me (1999)", 0.9579553195370458, 26]
"Outbreak (1995)"	["Children of Men (2006)", 0.9579878077949445, 16]
"Outbreak (1995)"	["Robin Hood: Men in Tights (1993)", 0.958084171388575, 28]
"Outbreak (1995)"	["Jumanji (1995)", 0.9581860871374344, 42]
"Outbreak (1995)"	["Grumpier Old Men (1995)", 0.9582209228064007, 20]
"Outbreak (1995)"	["My Big Fat Greek Wedding (2002)", 0.9582353240450506, 13]
"Outbreak (1995)"	["War of the Worlds (2005)", 0.9583804959174713, 12]
"Outbreak (1995)"	["Patriot Games (1992)", 0.9584250559231255, 12]
"Outbreak (1995)"	["Mary Poppins (1964)", 0.958649627361426, 14]
"Outbreak (1995)"	["Star Wars: Episode III - Revenge of the Sith (2005)", 0.9586802002987105, 14]
"Outbreak (1995)"	["Rumble in the Bronx (Hont faan kui) (1995)", 0.9587160530770674, 19]
"Outbreak (1995)"	["Titanic (1997)",

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
! head -n 20 ratings.csv > test.csv